<a href="https://colab.research.google.com/github/aregeezra/EnergyAnomaly/blob/main/EnergyAnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 Installing necessary packages

# 0.0 IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays

from IPython.core.display  import HTML
from IPython.display       import Image
from datetime              import *
from tabulate              import tabulate
from scipy.stats           import chi2_contingency

from boruta                import BorutaPy
from sklearn.ensemble      import RandomForestRegressor

from sklearn.metrics       import mean_absolute_error, mean_squared_error
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.ensemble      import RandomForestRegressor
import xgboost as xgb

import missingno as msno
import statsmodels.api as sm

import random
import warnings
warnings.filterwarnings( 'ignore' )

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:

def cramer_v( x, y ):
    cm = pd.crosstab( x, y ).values
    n = cm.sum()
    r, k = cm.shape
    
    chi2 = chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)

    return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )

def settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [ ]:
settings()

Populating the interactive namespace from numpy and matplotlib


# 0.2 Loading data

In [ ]:
train_features = pd.read_csv('./train_features.csv',on_bad_lines='skip')

FileNotFoundError: ignored

In [ ]:
train_features.sample(5)

# 1.0 Data Description

## 1.1 Data Dimensions

In [ ]:
print(f'Number of rows:{train_features.shape[0]}')
print(f'Number of columns:{train_features.shape[1]}')

# 1.3 Data Types

In [ ]:
pd.DataFrame({"Data Types": train_features.dtypes})

# 1.4 Check NA

In [ ]:
train_features.isna().sum()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(train_features.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
pd.DataFrame({"Missing Values(%)":
              train_features.isna().sum()/len(train_features.index)*100})

# 1.5 Fillout NA

In [ ]:
mean1 = train_features['meter_reading'].mean()

train_features['meter_reading'].fillna(value=mean1, inplace =True)


# 1.6 Change Types

In [ ]:
train_features.dtypes

In [ ]:

train_features['timestamp'] = pd.to_datetime(train_features['timestamp'])

# 1.6 Descriptive Statistical

In [ ]:
train_features.sample()

In [ ]:
num1 = train_features.select_dtypes(include = ['float64'])
cat1 = train_features.select_dtypes(exclude = ['float64', 'int64', 'datetime64[ns]'])

In [ ]:
num1.sample()

In [ ]:

cat1.sample()

## 1.6.1 Numerical Attributes

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame( num1.apply( np.mean ) ).T
ct2 = pd.DataFrame( num1.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num1.apply( np.std ) ).T
d2 = pd.DataFrame( num1.apply( min ) ).T
d3 = pd.DataFrame( num1.apply( max ) ).T
d4 = pd.DataFrame( num1.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num1.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num1.apply( lambda x: x.kurtosis() ) ).T
d7 = pd.DataFrame( num1.apply(lambda x: x.var())).T
# concatenate
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6, d7] ).T.reset_index()
m.columns = ( ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis', 'Variance'])


In [ ]:
m

## 1.6.2 Categorical Attributes

In [ ]:
cat1.apply( lambda x: x.unique().shape[0] )

In [ ]:
df1 = train_features.copy()

In [ ]:
plt.subplot(1, 2, 1)
sns.boxplot( x= 'anomaly', y='meter_reading' , data=df1 )

plt.subplot(1, 2, 2)
sns.boxplot( x= 'primary_use', y='meter_reading' , data=df1 )
plt.xticks(rotation=90)


# 2.0 FEATURE ENGINEERING

In [ ]:
df2 = df1.copy()

In [ ]:
df2['timestamp'].sample()

In [ ]:
df2.columns

In [ ]:
df2['date'] = df2['timestamp'].dt.date

df2['hour_of_day'] = df2['timestamp'].dt.hour

df2['day_of_week'] = df2['timestamp'].dt.weekday

df2['month'] = df2['timestamp'].dt.month


In [ ]:
df2.sample(10).T

# 3.0 EXPLORATORY DATA ANALYSIS

In [ ]:
df3 = df2.copy()

## 3.1 Univariate Analysis

### 3.1.1 Response Variable

In [ ]:
sns.distplot(df3['meter_reading'])

### 3.1.2 Numerical Variable

In [ ]:
plt.figure(figsize=(24,15))
num1.hist(bins=25)

### 3.1.3 Categorical Variable


In [ ]:
cat1.sample()

In [ ]:
df3.primary_use.unique()

In [ ]:
plt.subplot(1,2,1)
sns.countplot(df3['primary_use'])
plt.xticks(rotation=90)

plt.subplot(1,2,2)
sns.kdeplot( df3[df3['primary_use']=='Education']['meter_reading'], label='Education')
sns.kdeplot( df3[df3['primary_use']=='Office']['meter_reading'], label='Office')
sns.kdeplot( df3[df3['primary_use']=='Parking']['meter_reading'], label='Parking')
sns.kdeplot( df3[df3['primary_use']=='Lodging/residential']['meter_reading'], label='Lodging')
sns.kdeplot( df3[df3['primary_use']=='Entertainment/public assembly']['meter_reading'], label='Entertainment')
sns.kdeplot( df3[df3['primary_use']=='Public services']['meter_reading'], label='Public Service')
sns.kdeplot( df3[df3['primary_use']=='Manufacturing/industrial']['meter_reading'], label='Manufacturing/industrial')
sns.kdeplot( df3[df3['primary_use']=='Services']['meter_reading'], label='Services')
sns.kdeplot( df3[df3['primary_use']=='Other']['meter_reading'], label='Other')
sns.kdeplot( df3[df3['primary_use']=='Healthcare']['meter_reading'], label='Healthcare')
sns.kdeplot( df3[df3['primary_use']=='Food sales and service']['meter_reading'], label='FoodSale')
sns.kdeplot( df3[df3['primary_use']=='Religious worship']['meter_reading'], label='Religious worship')
plt.legend()
plt.show()

## 3.2 Bivariate Analysis

#### Percentage of Building with Anomalies


In [ ]:

df3['building_id'] = df3['building_id'].astype(str)


In [ ]:


anomaly = df3[['building_id', 'anomaly']]

In [ ]:
anomaly.columns = ['Buildings', "Anomaly"]

In [ ]:

def plot_stacked_bars(dataframe, title_, size_=(18, 10), rot_=0, legend_="upper right"):

  """
  Plot stacked bars with annotations
  """
  ax = dataframe.plot(kind="bar",
                    stacked=True,
                    figsize=size_,
                    rot=rot_,
                    title=title_)
  # Annotate bars
  annotate_stacked_bars(ax, textsize=14)
  # Rename legend
  plt.legend(["Normal", "Anomaly"], loc=legend_)
  # Labels
  plt.ylabel("Building base (%)")
  plt.show()

def annotate_stacked_bars(ax, pad=0.99, colour="white", textsize=13):
  """
  Add value annotations to the bars
  """
  # Iterate over the plotted rectanges/bars
  for p in ax.patches:
    # Calculate annotation
    value = str(round(p.get_height(),1))
    # If value is 0 do not annotate
    if value == '0.0':
     continue
    ax.annotate(value,
                ((p.get_x()+ p.get_width()/2)*pad-0.05, (p.get_y()+p.get_height()/2)*pad),
                color=colour,
                size=textsize,
    )


In [ ]:
anomaly_total = anomaly.groupby(anomaly['Anomaly']).count()
anomaly_percentage = anomaly_total / anomaly_total.sum() * 100

In [ ]:
plot_stacked_bars(anomaly_percentage.transpose(),"Building status", (5,5), legend_="lower right")

In [ ]:
sns.lineplot(data=df3, x="building_id", y="meter_reading", hue="anomaly",palette=['b', 'r'])
plt.xticks(rotation=90)


Perecentage of Building with Anomalies categorized by the primary use

In [ ]:
sector = df3[['building_id', 'primary_use', 'anomaly']]
sector = sector.groupby([sector['primary_use'], sector['anomaly']])['building_id'].count().unstack(level=1).fillna(0)
sector_anomaly= (sector.div(sector.sum(axis=1), axis=0) * 100).sort_values(by=[1], ascending=False)

In [ ]:
plot_stacked_bars(sector_anomaly, "Sector Anomaly", rot_=90)

In [ ]:
time = df3[['building_id', 'hour_of_day', 'anomaly']]
time = time.groupby([time['hour_of_day'], time['anomaly']])['building_id'].count().unstack(level=1).fillna(0)
time_anomaly= (time.div(time.sum(axis=1), axis=0) * 100).sort_values(by=[1], ascending=False)

In [ ]:
plot_stacked_bars(time_anomaly, "Time Anomaly", rot_=90)

In [ ]:
sns.lineplot(data=df3, x="hour_of_day", y="meter_reading", hue="anomaly")



In [ ]:
day_of_week= df3[['building_id', 'weekday', 'anomaly']]
day_of_week = day_of_week.groupby([day_of_week['weekday'], day_of_week['anomaly']])['building_id'].count().unstack(level=1).fillna(0)
day_anomaly= (day_of_week.div(day_of_week.sum(axis=1), axis=0) * 100).sort_values(by=[1], ascending=False)

In [ ]:
plot_stacked_bars(day_anomaly, "Day_Anomaly", rot_=90)

In [ ]:
sns.lineplot(data=df3, x="weekday", y="meter_reading", hue="anomaly")

In [ ]:
df3.columns

In [ ]:
floor= df3[['building_id', 'floor_count', 'anomaly']]
floor = floor.groupby([floor['floor_count'], floor['anomaly']])['building_id'].count().unstack(level=1).fillna(0)
floor_anomaly= (floor.div(floor.sum(axis=1), axis=0) * 100).sort_values(by=[1], ascending=False)

In [ ]:
plot_stacked_bars(floor_anomaly, "Floor Count Anomaly", rot_=90)

In [ ]:
df3.columns

In [ ]:
plt.figure(figsize=(20,54))
plt.subplot(6,1,1)
sns.scatterplot(data=df3, x="air_temperature", y="meter_reading",
                hue="anomaly", cmap=plt.cm.jet)

plt.subplot(6,1,2)
sns.scatterplot(data=df3, x="cloud_coverage", y="meter_reading",
                hue="anomaly", cmap=plt.cm.jet)

plt.subplot(6,1,3)
sns.scatterplot(data=df3, x="dew_temperature", y="meter_reading",
                hue="anomaly", cmap=plt.cm.jet)

plt.subplot(6,1,4)
sns.scatterplot(data=df3, x="sea_level_pressure", y="meter_reading",
                hue="anomaly", cmap=plt.cm.jet)

plt.subplot(6,1,5)
sns.scatterplot(data=df3, x="wind_direction", y="meter_reading",
                hue="anomaly", cmap=plt.cm.jet)

plt.subplot(6,1,6)
sns.scatterplot(data=df3, x="wind_speed", y="meter_reading",
                hue="anomaly", cmap=plt.cm.jet)

## 3.3 Multivariate Analysis


In [ ]:
plt.figure(figsize=(35, 45))
correlation = num1.corr( method='pearson' )
sns.heatmap( correlation, annot=True )

## 3.3.1 Prinicipal Component Analysis

Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. The number of distinct principal components is equal to the smaller of the number of original variables or the number of observations minus one. This transformation is defined in such a way that the first principal component has the largest possible variance (that is, accounts for as much of the variability in the data as possible), and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal the preceding components. The resulting vectors are an uncorrelated orthogonal basis set.

In [ ]:
df4 = df3.copy()

In [ ]:
df4.sample()

In [ ]:
df4.drop(['timestamp', 'primary_use',], axis = 1)